Fine-tuning with Python
----------------------------

The recommended way to do training is with the `main.py` script in ocp. One of the reasons for that is training often takes a long time and is better suited for queue systems like slurm. However, you can submit Python scripts too, and it is possible to run notebooks in Slurm too. Here we work out a proof of concept in training from Python and a Jupyter notebook.

In [1]:
%run ../ocp-tutorial.ipynb

In [2]:
import logging
from ocpmodels.common.utils import SeverityLevelBetween

root = logging.getLogger()

 
root.setLevel(logging.INFO)

log_formatter = logging.Formatter(
            "%(asctime)s (%(levelname)s): %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",)

# Send INFO to stdout
handler_out = logging.FileHandler('out.txt', 'w')
handler_out.addFilter(
            SeverityLevelBetween(logging.INFO, logging.WARNING)
        )
handler_out.setFormatter(log_formatter)
root.addHandler(handler_out)

# Send WARNING (and higher) to stderr
handler_err = logging.FileHandler('out.txt', 'w+')
handler_err.setLevel(logging.WARNING)
handler_err.setFormatter(log_formatter)
root.addHandler(handler_err)

In [3]:
! ase db ../fine-tuning/oxides.db

id| age|formula|calculator| energy|natoms| fmax|pbc| volume|charge|   mass
 1|224m|Sn2O4  |unknown   |-41.359|     6|0.045|TTT| 64.258| 0.000|301.416
 2|224m|Sn2O4  |unknown   |-41.853|     6|0.025|TTT| 66.526| 0.000|301.416
 3|224m|Sn2O4  |unknown   |-42.199|     6|0.010|TTT| 68.794| 0.000|301.416
 4|224m|Sn2O4  |unknown   |-42.419|     6|0.006|TTT| 71.062| 0.000|301.416
 5|224m|Sn2O4  |unknown   |-42.534|     6|0.011|TTT| 73.330| 0.000|301.416
 6|224m|Sn2O4  |unknown   |-42.562|     6|0.029|TTT| 75.598| 0.000|301.416
 7|224m|Sn2O4  |unknown   |-42.518|     6|0.033|TTT| 77.866| 0.000|301.416
 8|224m|Sn2O4  |unknown   |-42.415|     6|0.010|TTT| 80.134| 0.000|301.416
 9|224m|Sn2O4  |unknown   |-42.266|     6|0.006|TTT| 82.402| 0.000|301.416
10|224m|Sn2O4  |unknown   |-42.083|     6|0.017|TTT| 84.670| 0.000|301.416
11|224m|Sn4O8  |unknown   |-81.424|    12|0.012|TTT|117.473| 0.000|602.832
12|224m|Sn4O8  |unknown   |-82.437|    12|0.005|TTT|121.620| 0.000|602.832
13|224m|Sn4O8  |unknown  

In [4]:
checkpoint = get_checkpoint('GemNet-OC OC20+OC22')
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
calc = OCPCalculator(checkpoint=checkpoint, trainer='forces', cpu=False)

amp: true
cmd:
  checkpoint_dir: /home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/checkpoints/2023-08-01-17-18-56
  commit: 3973c79
  identifier: ''
  logs_dir: /home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/logs/tensorboard/2023-08-01-17-18-56
  print_every: 100
  results_dir: /home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/results/2023-08-01-17-18-56
  seed: null
  timestamp_id: 2023-08-01-17-18-56
dataset: null
gpus: 1
logger: tensorboard
model: gemnet_oc
model_attributes:
  activation: silu
  atom_edge_interaction: true
  atom_interaction: true
  cbf:
    name: spherical_harmonics
  cutoff: 12.0
  cutoff_aeaint: 12.0
  cutoff_aint: 12.0
  cutoff_qint: 12.0
  direct_forces: true
  edge_atom_interaction: true
  emb_size_aint_in: 64
  emb_size_aint_out: 64
  emb_size_atom: 256
  emb_size_cbf: 16
  emb_size_edge: 512
  emb_size_quad_in: 32
  emb_size_quad_out: 32
  emb_size_rbf: 16
  emb_size_sbf: 32
  emb_size_trip_in: 64
  e

## Split the data into train, test, val sets

In [5]:
! rm -fr train.db test.db val.db
train, test, val = train_test_val_split('../fine-tuning/oxides.db')
train, test, val

(PosixPath('/home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/train.db'),
 PosixPath('/home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/test.db'),
 PosixPath('/home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/val.db'))

# Setup the training code

We start by making the config.yml. We build this from the calculator checkpoint.

In [6]:
yml = generate_yml_config(checkpoint, 'config.yml',
                   delete=['slurm', 'cmd', 'logger', 'task', 'model_attributes',
                           'optim.loss_force', # the checkpoint setting causes an error
                           'dataset', 'test_dataset', 'val_dataset'],
                   update={'gpus': 1,
                           'task.dataset': 'ase_db',
                           'optim.eval_every': 1,
                           'optim.max_epochs': 5,
                           # Train data
                           'dataset.train.src': 'train.db',
                           'dataset.train.a2g_args.r_energy': True,
                           'dataset.train.a2g_args.r_forces': True,
                            # Test data - prediction only so no regression
                           'dataset.test.src': 'test.db',
                           'dataset.test.a2g_args.r_energy': False,
                           'dataset.test.a2g_args.r_forces': False,
                           # val data
                           'dataset.val.src': 'val.db',
                           'dataset.val.a2g_args.r_energy': True,
                           'dataset.val.a2g_args.r_forces': True,
                          })

yml

PosixPath('/home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/config.yml')

## Setup the training task

This essentially allows several opportunities to define and override the config. You start with the base config.yml, and then via "command-line" arguments you specify changes you want to make. 

The code is build around `submitit`, which is often used with Slurm, but also works locally.

We have to mimic the `main.py` setup to get the arguments and config setup. Here is a minimal way to do this.

In [7]:
from ocpmodels.common.flags import flags
parser = flags.get_parser()
args, args_override = parser.parse_known_args(["--mode=train",                                            
                                               "--config-yml=config.yml", 
                                               f"--checkpoint={checkpoint}",
                                               "--amp"])
args, args_override

(Namespace(mode='train', config_yml=PosixPath('config.yml'), identifier='', debug=False, run_dir='./', print_every=10, seed=0, amp=True, checkpoint='gnoc_oc22_oc20_all_s2ef.pt', timestamp_id=None, sweep_yml=None, submit=False, summit=False, logdir=PosixPath('logs'), slurm_partition='ocp', slurm_mem=80, slurm_timeout=72, num_gpus=1, distributed=False, cpu=False, num_nodes=1, distributed_port=13356, distributed_backend='nccl', local_rank=0, no_ddp=False, gp_gpus=None),
 [])

Next, we build the first stage in our config. This starts with the file config.yml, then updates it with the args

In [8]:
from ocpmodels.common.utils import build_config, new_trainer_context

config = build_config(args=args, args_override={})
config

{'amp': True,
 'checkpoint': 'gnoc_oc22_oc20_all_s2ef.pt',
 'dataset': {'test': {'a2g_args': {'r_energy': False, 'r_forces': False},
   'src': 'test.db'},
  'train': {'a2g_args': {'r_energy': True, 'r_forces': True},
   'src': 'train.db'},
  'val': {'a2g_args': {'r_energy': True, 'r_forces': True}, 'src': 'val.db'}},
 'gpus': 1,
 'model': {'activation': 'silu',
  'atom_edge_interaction': True,
  'atom_interaction': True,
  'cbf': {'name': 'spherical_harmonics'},
  'cutoff': 12.0,
  'cutoff_aeaint': 12.0,
  'cutoff_aint': 12.0,
  'cutoff_qint': 12.0,
  'direct_forces': True,
  'edge_atom_interaction': True,
  'emb_size_aint_in': 64,
  'emb_size_aint_out': 64,
  'emb_size_atom': 256,
  'emb_size_cbf': 16,
  'emb_size_edge': 512,
  'emb_size_quad_in': 32,
  'emb_size_quad_out': 32,
  'emb_size_rbf': 16,
  'emb_size_sbf': 32,
  'emb_size_trip_in': 64,
  'emb_size_trip_out': 64,
  'envelope': {'exponent': 5, 'name': 'polynomial'},
  'extensive': True,
  'forces_coupled': False,
  'max_neigh

# Run the training task

It is still annoying that if your output is too large the notebook will not be able to be saved. On the other hand, it is annoying to simply capture the output. 

We are able to redirect most logging to a file above, but not all of it. The link below will open the file in a browser, and the subsequent cell captures all residual output. We do not need any of that, so it is ultimately discarded. 

Alternatively, you can open a Terminal and use `tail -f out.txt` to see the progress.

In [9]:
from IPython.display import display, FileLink
display(FileLink('out.txt'))

/home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/advanced/out.txt

In [13]:
with new_trainer_context(config=config, args=args) as ctx:
    config = ctx.config
    task = ctx.task
    trainer = ctx.trainer
    task.setup(trainer)
    task.run()

amp: true
cmd:
  checkpoint_dir: ./checkpoints/2023-08-01-17-42-24
  commit: 3973c79
  identifier: ''
  logs_dir: ./logs/tensorboard/2023-08-01-17-42-24
  print_every: 10
  results_dir: ./results/2023-08-01-17-42-24
  seed: 0
  timestamp_id: 2023-08-01-17-42-24
dataset:
  a2g_args:
    r_energy: true
    r_forces: true
  src: train.db
gpus: 1
logger: tensorboard
model: gemnet_oc
model_attributes:
  activation: silu
  atom_edge_interaction: true
  atom_interaction: true
  cbf:
    name: spherical_harmonics
  cutoff: 12.0
  cutoff_aeaint: 12.0
  cutoff_aint: 12.0
  cutoff_qint: 12.0
  direct_forces: true
  edge_atom_interaction: true
  emb_size_aint_in: 64
  emb_size_aint_out: 64
  emb_size_atom: 256
  emb_size_cbf: 16
  emb_size_edge: 512
  emb_size_quad_in: 32
  emb_size_quad_out: 32
  emb_size_rbf: 16
  emb_size_sbf: 32
  emb_size_trip_in: 64
  emb_size_trip_out: 64
  envelope:
    exponent: 5
    name: polynomial
  extensive: true
  forces_coupled: false
  max_neighbors: 30
  max_nei

/home/jovyan/shared-scratch/jkitchin/tutorial/ocp-tutorial/fine-tuning/ocp/ocpmodels/datasets/ase_datasets.py:108: UserWarning: Supplied sid is not numeric (or missing). Using dataset indices instead.
  warnings.warn(
device 0: 100%|██████████| 2/2 [00:00<00:00,  5.95it/s]


In [12]:
! head out.txt
! tail out.txt

2023-08-01 17:18:26 (WARNING): Unrecognized arguments: ['symmetric_edge_symmetrization']
2023-08-01 17:18:34 (WARNING): Unable to identify OCP trainer, defaulting to `forces`. Specify the `trainer` argument into OCPCalculator if otherwise.
2023-08-01 17:18:34 (WARNING): Unrecognized arguments: ['symmetric_edge_symmetrization']
2023-08-01 17:18:38 (WARNING): Unrecognized arguments: ['symmetric_edge_symmetrization']
2023-08-01 17:18:41 (WARNING): Model gradient logging to tensorboard not yet supported.
pretrained checkpoint, this is the correct behavior. To disable this, delete `scale_dict` from the checkpoint. 
2023-08-01 17:18:34 (INFO): Loading dataset: oc22_lmdb
2023-08-01 17:18:34 (INFO): Loading model: gemnet_oc
2023-08-01 17:18:37 (INFO): Loaded GemNetOC with 38864438 parameters.
2023-08-01 17:18:37 (INFO): Loading checkpoint from: gnoc_oc22_oc20_all_s2ef.pt
2023-08-01 17:20:21 (INFO): forcesx_mae: 0.0091, forcesy_mae: 0.0103, forcesz_mae: 0.0080, forces_mae: 0.0091, forces_cos: 0

Now, you are all set to carry on with what ever subsequent analysis you want to do. 